## Variables to set

In [ ]:
# set the model type
estimModel = 'hurdle' #nbinom or hurdle

# list of the (prediction) windows
max_w = 6 #36
window_list = list(range(2, max_w+1))

# remove countries
removeCountries = True
lastX = 2 # all but the last x countries are removed

### google drive runtime connection

In [ ]:
# Ctrl + F9 alle Zellen ausführen

!pip install PyDrive
!pip install CRPS

In [ ]:
import pyarrow.parquet as pq
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth) 

data_folder_id = '1RigGnEyyNGnO_SPBSc_RwO9jjdbnPTAV'
result_folder_id = '1CNBTHtBOTFXh01WUpP2EF1aEmDi0rSyg'  

### Import packages and define functions

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import CRPS.CRPS as pscore
import copy
from joblib import dump, load, Parallel, delayed
import statsmodels.api as sm
from scipy.stats import nbinom, poisson
from time import sleep
from tqdm import tqdm

## functions for the distribtion models
# truncated negative binomial---------------------------------------------------
def truncNegBin_CDF(y, n, p):
    f_zero = nbinom.pmf(0, n, p)
    if y > 0:
        return (nbinom.cdf(y, n, p) - nbinom.cdf(0, n, p)) / (1 - f_zero)
    else:
        return 0
    
def truncNegBin_PPF(x, n, p, epsilon=1e-6, max_iterations=100):
    # if f(0)=0 no truncation is needed
    if (1 - nbinom.pmf(0, n, p)) == 1:
        return nbinom.ppf(x, n, p)
    else:
        # Define the range of y where the solution might exist
        lower_bound = 0
        upper_bound = 1000000000  # Adjust this based on the expected range of y

        # Bisection method
        for _ in range(max_iterations):
            y = (lower_bound + upper_bound) / 2
            cdf_value = truncNegBin_CDF(y, n, p)

            if abs(cdf_value - x) < epsilon:
                return np.ceil(y)  # Found a good approximation

            if cdf_value < x:
                lower_bound = y
            else:
                upper_bound = y

        # Return the best approximation if max_iterations is reached
        return np.ceil(y)

def calculate_trunc_nbinom_quantile(quantile, n, p):
    return truncNegBin_PPF(quantile, n, p)

# truncated poisson---------------------------------------------------
def truncPois_CDF(y, mu):
    f_zero = poisson.pmf(0, mu)
    if y > 0:
        return (poisson.cdf(y, mu) - poisson.cdf(0, mu)) / (1 - f_zero)
    else:
        return 0
    
def truncPois_PPF(x, mu, epsilon=1e-6, max_iterations=100):
    # if f(0)=0 no truncation is needed
    if (1 - poisson.pmf(0, mu)) == 1:
        return poisson.ppf(x, mu)
    else:
        # Define the range of y where the solution might exist
        lower_bound = 0
        upper_bound = 1000000000  # Adjust this based on the expected range of y

        # Bisection method
        for _ in range(max_iterations):
            y = (lower_bound + upper_bound) / 2
            cdf_value = truncPois_CDF(y, mu)

            if abs(cdf_value - x) < epsilon:
                return np.ceil(y)  # Found a good approximation

            if cdf_value < x:
                lower_bound = y
            else:
                upper_bound = y

        # Return the best approximation if max_iterations is reached
        return np.ceil(y)

def calculate_trunc_pois_quantile(quantile, mu):
    return truncPois_PPF(quantile, mu)

### function to compute distribution-------------------------------------------------
def baseFatalModel_quantiles(featureSeries, quantiles, w=None, model='hurdle'):
    # list to store quantiles 
    dummy_fatalities_list = []
    # string to store model distribution
    dist_string = ''

    mean = None
    var = None

    numberQuantiles = len(quantiles)

    # hurdle model
    if model == 'hurdle':
        if w == None:
            
            # calculate pt, i.e. the probability that y>0
            p_t = 1 - (featureSeries.value_counts().get(0, 0) / featureSeries.count())
            # calculate n (r) and p via average/variance without the zero values
            mean = pd.Series.mean(featureSeries[featureSeries != 0])
            var = pd.Series.var(featureSeries[featureSeries != 0])

        elif w <= 0:
            return 'w has to be > 0'
        
        else:
            features = featureSeries.tail(w).loc[:,'ged_sb']
            # calculate pt, i.e. the probability that y>0
            p_t = 1 - (features.value_counts().get(0, 0) / features.count())
            # calculate n (r) and p via average/variance without the zero values
            mean = pd.Series.mean(features[features != 0])
            var = pd.Series.var(features[features != 0])

        # pd.Series.var or mean returns Nan in case of a passed series of length 1
        if np.isnan(mean):
            mean = 0
        if np.isnan(var):
            var = 0

        # check if there are values above zero, otherwise no second component (trunc dist.) needed
        if p_t > 0:
            # component 1, y=0: Bernoulli
            comp2_quantiles = [q for q in quantiles if q > p_t] #quantiles for the second component
            removed_elements_length = numberQuantiles-len(comp2_quantiles)
            zeros_array = np.zeros(removed_elements_length) #zero values that originate from the bernoulli dist

            # component 2, y>0
            if var != 0 and var > mean:
                n = (mean**2) / (var - mean) # equivalent to r
                p = mean / var
                trunc_nbinom_quantiles = Parallel(n_jobs=-1)(delayed(calculate_trunc_nbinom_quantile)(quantile, n, p) for quantile in comp2_quantiles) #fast way
                trunc_nbinom_quantiles = np.array(trunc_nbinom_quantiles)

                dummy_fatalities_list = np.concatenate((zeros_array, trunc_nbinom_quantiles)).tolist()
                dist_string = 'BernoulliTruncNBinom'

            elif mean == 0 and var == 0: # due to faster calculation
                dummy_fatalities_list = [0] * numberQuantiles
                dist_string = 'BernoulliTruncPois'

            else:  # equivalent to all means and 0 < var <= mean
                trunc_pois_quantiles = Parallel(n_jobs=-1)(delayed(calculate_trunc_pois_quantile)(quantile, mean) for quantile in comp2_quantiles) #fast way
                trunc_pois_quantiles = np.array(trunc_pois_quantiles)
                
                dummy_fatalities_list = np.concatenate((zeros_array, trunc_pois_quantiles)).tolist()
                dist_string = 'BernoulliTruncPois'
            
        # p_t = 0 so no second component is needed    
        else:
            dummy_fatalities_list = [0] * numberQuantiles
            dist_string = 'BernoulliHurdle'

    # nbinom model
    elif model == 'nbinom':
        if w == None:
             # calculate n (r) and p via average/variance
            mean = pd.Series.mean(featureSeries)
            var = pd.Series.var(featureSeries)
        elif w <= 0:
            return 'w has to be > 0'
        else:
            # calculate n (r) and p via average/variance
            mean = pd.Series.mean(featureSeries.tail(w).loc[:,'ged_sb'])
            var = pd.Series.var(featureSeries.tail(w).loc[:,'ged_sb'])

        if var != 0 and var > mean:
                n = (mean**2) / (var - mean) # equivalent to r
                p = mean / var
                dummy_fatalities_list = nbinom.ppf(quantiles, n, p).tolist()
                dist_string = 'NBinom'

        elif mean == 0 and var == 0: # due to faster calculation
                dummy_fatalities_list = [0] * numberQuantiles
                dist_string = 'Pois'

        else: # equivalent to all means and 0 < var <= mean
                dummy_fatalities_list = poisson.ppf(quantiles, mean).tolist()
                dist_string = 'Pois'

    return {'fatalities': dummy_fatalities_list, 'dist': dist_string, 'mean': mean, 'var': var}

In [ ]:
## import files-----------------------------
# Liste der Dateien im Ordner
file_list = drive.ListFile({'q': f"'{data_folder_id}' in parents and trashed=false"}).GetList()

# create the feature- and actuals-data list
# set the feature and actuals year lists
feature_years = ['2017','2018','2019','2020']
actual_years = ['2018','2019','2020','2021']

actuals_df_list = []
features_df_list = []

# store data from google drive in lists
for i in range(len(feature_years)):

    feature_title = 'cm_features_to_oct' + feature_years[i] + '.parquet'

    for file in file_list:
        if file['title'] == feature_title:
            file.GetContentFile(file['title'])
            parquet_file = pq.ParquetFile(file['title'])
            #loaded_data[file['title']] = parquet_file.read().to_pandas()

            features_df_list.append({'year':feature_years[i], 'data':parquet_file.read().to_pandas()})

        actual_title = 'cm_actuals_' + actual_years[i] + '.parquet'

        if file['title'] == actual_title:
            file.GetContentFile(file['title'])
            parquet_file = pq.ParquetFile(file['title'])
            #loaded_data[file['title']] = parquet_file.read().to_pandas()

            actuals_df_list.append({'year':actual_years[i], 'data':parquet_file.read().to_pandas()})

# concat the feature datasets, so that every data contains the observations starting with january 1990
for i in range(1,len(features_df_list)):
    features_df_list[i]['data'] = pd.concat([features_df_list[i-1]['data'], features_df_list[i]['data']])

In [ ]:
# function to check, if the last n months are in the dataset of a country,
# other than that the last month of a country in the feature dataset has to be 3 months before the first actuals month!!
def check_last_nMonths(n, country_id, yearindex):
    country = country_feature_group_list[yearindex].get_group(country_id)

    # reference month of the actual dataset
    actual_month_list = actuals_df_list[yearindex]['data'].index.get_level_values('month_id').unique().tolist()

    # if the last month of the feature dataset in the country does not match the first of the actuals return false
    if (actual_month_list[0] - 3) != country.index.get_level_values('month_id').unique().tolist()[-1]:
        return False
    else:
        month_list = features_df_list[yearindex]['data'].index.get_level_values('month_id').unique().tolist()
        last_month = month_list[-1] # equals the first month - 3 from the corresponding actuals dataset
        first_month = month_list[0]

        last_n_months = True

        if last_month-n+1 < first_month:
            last_n_months = False
        else:
            month_list = list(range(last_month-n+1, last_month+1))
            
            for month in month_list:
                if month not in country.index.get_level_values('month_id'):
                    last_n_months = False
                    break

        return last_n_months
        #return True

#--------------------------------------------------------------------------------------------
# because of the concatination only the last dataframe is used (later on the appended months are dropped for datasets before 2020)

# IF THIS CODE IS USED FOR THE 2024 PREDICTION ADJUST THE features_1990to2020 in the whole file to 1990to2023

features_1990to2020_df = features_df_list[3]['data']
country_list = sorted(features_df_list[3]['data'].index.get_level_values('country_id').unique().tolist())

# country group list of all four datasets
country_feature_group_list = []
country_actual_group_list = []
# fill list 
for i in range(len(features_df_list)):
    country_feature_group_list.append(features_df_list[i]['data'].groupby('country_id'))
    country_actual_group_list.append(actuals_df_list[i]['data'].groupby('country_id'))

""" # same reason as mentioned two lines earlier
country_feature_group_1990to2020 = country_feature_group_list[3] """


print(len(country_list))

In [ ]:
# modify country_list so that it contains only country_ids 
# that have at least the last n months of observations in the last dataset (2020)!
numberMonths_toOct20 = 96 # 96 = 5*12 (5 jahre für 2017) + 3*12 (jedes Jahr 12 Monate mehr also 2020 8 Jahre)
#ABER: um konsistent zu bleiben wird für jedes Jahr (jeden to_octX Datensatz) nur die letzten 5 Jahre verwendet!!!

#-- note------
# dataset 2020 is used, because of the structure of the other datasets.
# 2020 is dataset 2019 with 12 additional rows (months) etc.
# for the CRPS calculation  of the datasets != 2020 the last 12*x windows are deleted
# this procedure is saving computation time
#-------------


#IMPORTANT
#if you do not minimize over all countries but only the single countries, 
# it is sufficient to check if all countries contain the last month in the features dataset (this way you use the full information). 
# But you still have to check check_last_nMonths(len(countrymonths), countryIndex, 3), so that no month is missing in between.

# => so currently not all information is used for each country

dummy_list = []
for countryIndex in country_list:
    dummy_hasLastN_months = True

    # index 3 is the last dataset
    # 76, da Land 246 z.b. genau die letzten 112 Monate (in '2020') als Beobachtungen hat 
    if check_last_nMonths(numberMonths_toOct20, countryIndex, 3) is not True:
        dummy_hasLastN_months = False  
    
    if dummy_hasLastN_months is True:
        dummy_list.append(countryIndex)

# the values in country_list are the 'country_id'
country_list = dummy_list

#IMPORTANT
# all countries that have the last month as observation have the last 96 months as observations (in 2020)!!! so no country is excluded
# checked by modifing the check_last_nMonths function -> else: return True

len(country_list)

## Baseline 1-4

Optimize **w** (through the CRPS) regarding
|            | datasets    | countries   | prediction windows |
|------------|-------------|-------------|--------------------|
| baseline 1 | all         | all         | all                |
| baseline 2 | all         | inidvidual  | all                |
| baseline 3 | all         | all         | individual         |
| baseline 4 | all         | inidvidual  | individual         |

#### The minimization is based on calculating the quantiles for each country, w and year (of the four datasets).

In [ ]:
## changes, so that the calculation does not take a long time -------------------
if removeCountries == True:
    # remove all but last x countries
    elements_to_remove = country_list[0:(len(country_list)-lastX)] # only last x countries
    country_list = [element for element in country_list if element not in elements_to_remove]
    
len(country_list)

In [ ]:
s_prediction_list = list(range(3, 15)) # months to predict in the future

number_countries = len(country_list)
number_dataframes = len(actual_years)
number_w = len(window_list)

# lists for the estimation of the distribution
quantiles = np.arange(0.001, 0.9999, 0.001)
quantiles = [round(q, 3) for q in quantiles] # due to binary inaccuracies
string_quantile_list = [f"{round(q * 100, 1)}%" for q in quantiles] # sting values of the quantiles

# last month of the dataframe as reference for the moving prediction windows
last_month = features_df_list[3]['data'].index.get_level_values('month_id').tolist()[-1]

# list to store the estimations/predictions for each w
baseline_estimate_list = []

# loop through windows
for i in tqdm(range(number_w)):    
    sleep(3)
    #print('                              window ' + str(i+1) + '/' + str(number_w)  , end='\r')

    w = window_list[i] # current window
    baseline_estimate_list.append({'window':w, 
                                'country_predict_list':[{'country_id':country, 'predictionWindowsN':[]} for country in country_list]})
    
    #calculate the number of subsets, that are used to estimate the distribution and validate it via 12 months of actuals 
    # the number is dependent of the actual w. E.g. with the maximal w (e.g. 24): if w=24, actuals are 12 months (starting with s=3 to s=14) 
    # -> 24 + 2 + 12 = 39 observations of ged_sb per window
    # so if the dataset has 96 observations there are 96 - 38 = 58 shiftable windows for 2020
    numberWindows = numberMonths_toOct20 - (w + 2 + 12)

    windowLength = w + 2 + 12 # length of the individual window for the current w
    
    # loop through all countries
    for index in range(number_countries):
        country = country_list[index]
    
        #print('country ' + str(index+1) + '/' + str(number_countries), end='\r')

        features = country_feature_group_list[3].get_group(country) # features of country
        

        # loop through all X equal parts of the feature dataset (traindata length w, actuals is vector of the next t+3 till t+12 observations)
        for j in range(numberWindows):
            starting_month_window = last_month - windowLength + 1 - numberWindows + 1  + j
            ending_month_window = starting_month_window + w - 1

            starting_month_actuals = ending_month_window + 3
            ending_month_actuals = starting_month_actuals + 11
            
            window_features = features.loc[(slice(starting_month_window, ending_month_window), slice(None)), 'ged_sb']
            window_actuals = features.loc[(slice(starting_month_actuals, ending_month_actuals), slice(None)), 'ged_sb']

            #predict = nBinom_quantiles(window_features, 'None', quantiles)
            predict = baseFatalModel_quantiles(window_features, quantiles, model=estimModel)
            
            baseline_estimate_list[i]['country_predict_list'][index]['predictionWindowsN'].append(
                [{'country_id':country, 'w':w, 'dist':predict['dist'], 
                'mean':predict['mean'], 'var':predict['var'], 'first_month_feature':starting_month_window, 
                'quantile':string_quantile_list, 'fatalities':predict['fatalities']}, 
                {'s':s_prediction_list, 
                    'month_id': window_actuals.index.get_level_values('month_id'),
                    'unreal_actuals':window_actuals.values}])

In [ ]:
# save variables in joblib files
dump([actual_years,
      feature_years,
      estimModel,
      s_prediction_list,
      window_list,
      country_list,
      number_countries,
      number_dataframes,
      number_w,
      baseline_estimate_list], 
      'task2_baseline_estim_vars.joblib')

## upload files---------------------------
file1 = drive.CreateFile({'parents':[{u'id': result_folder_id}]})
file1.SetContentFile('task2_baseline_estim_vars.joblib')
file1.Upload()